In [1]:
dataframe = spark.read.option("inferschema", "true").option("header", "true").csv("flight06.csv")
dataframe.show(5)

+---+-----------+-------------+----------+-------+------+---------+----+-------+------+-------------+-----------------+--------------+--------------+
|_c0|     ItinID|        MktID|MktCoupons|Quarter|Origin|OriginWac|Dest|DestWac| Miles|ContiguousUSA|NumTicketsOrdered|AirlineCompany|PricePerTicket|
+---+-----------+-------------+----------+-------+------+---------+----+-------+------+-------------+-----------------+--------------+--------------+
|  0|20181767585|2018176758501|         1|      1|   PHL|       23| LAX|     91|2402.0|            2|              1.0|            AA|        672.87|
|  1|20181767586|2018176758601|         1|      1|   PHL|       23| LAX|     91|2402.0|            2|              1.0|            AA|        367.68|
|  2|20181767587|2018176758701|         1|      1|   PHL|       23| LAX|     91|2402.0|            2|              1.0|            AA|        417.94|
|  3|20181767636|2018176763601|         1|      1|   PHL|       23| LAX|     91|2402.0|            2

In [2]:
dataframe.groupBy("Dest").sum("miles").withColumnRenamed("sum(miles)", "destination_total").sort("destination_total").limit(5).explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#133 ASC NULLS FIRST], output=[Dest#23,destination_total#133])
+- *(2) HashAggregate(keys=[Dest#23], functions=[sum(miles#25)])
   +- Exchange hashpartitioning(Dest#23, 200), true, [id=#50]
      +- *(1) HashAggregate(keys=[Dest#23], functions=[partial_sum(miles#25)])
         +- FileScan csv [Dest#23,Miles#25] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Dawood Rana/Big Data/flight06.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Dest:string,Miles:double>




In [3]:
dataframe = spark.read.option("inferschema", "true").option("header", "true").csv("summary06.csv")
dataframe.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [4]:
# Question 01
from pyspark.sql.functions import col, sum
dataframe.where(col("DEST_COUNTRY_NAME")=="United States").groupBy("ORIGIN_COUNTRY_NAME").agg(sum("count")).sort("ORIGIN_COUNTRY_NAME").explain()

== Physical Plan ==
*(3) Sort [ORIGIN_COUNTRY_NAME#155 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(ORIGIN_COUNTRY_NAME#155 ASC NULLS FIRST, 200), true, [id=#118]
   +- *(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#155], functions=[sum(cast(count#156 as bigint))])
      +- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#155, 200), true, [id=#114]
         +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#155], functions=[partial_sum(cast(count#156 as bigint))])
            +- *(1) Project [ORIGIN_COUNTRY_NAME#155, count#156]
               +- *(1) Filter (isnotnull(DEST_COUNTRY_NAME#154) AND (DEST_COUNTRY_NAME#154 = United States))
                  +- FileScan csv [DEST_COUNTRY_NAME#154,ORIGIN_COUNTRY_NAME#155,count#156] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#154), (DEST_COUNTRY_NAME#154 = United States)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/Dawood Rana/Big Data/summary06.csv], PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NA

In [5]:
dataframe = spark.read.option("inferschema", "true").option("header", "true").csv("flight06.csv")
dataframe.groupBy("Dest").agg(sum("miles")).show()

+----+------------+
|Dest|  sum(miles)|
+----+------------+
| PSE|   3157476.0|
| DLG|     26978.0|
| MSY|1.21232583E8|
| PPG|    862868.0|
| GEG| 1.5338492E7|
| SNA| 7.3301545E7|
| BUR| 2.4765359E7|
| GRB|    227066.0|
| GTF|   1253347.0|
| IDA|    910594.0|
| GRR| 1.4140422E7|
| LWB|       489.0|
| PVU|    674614.0|
| PSG|    226663.0|
| EUG|   2720454.0|
| PVD| 2.6924425E7|
| GSO|   1940195.0|
| MYR|   8359385.0|
| OAK|1.10610381E8|
| MSN|   5571200.0|
+----+------------+
only showing top 20 rows



In [6]:
# Question 02: count total destination of flights
dataframe.groupBy("Dest").agg(sum("miles")).withColumnRenamed("sum(miles)","destination_total").orderBy("destination_total").limit(5).show()

+----+-----------------+
|Dest|destination_total|
+----+-----------------+
| ITH|            172.0|
| LAN|            179.0|
| MBS|            222.0|
| DRO|            250.0|
| LWB|            489.0|
+----+-----------------+

